In [ ]:
# Now we want to actually use the pipeline to calibrate an APERTIF observation
# Let's start the pipeline again as we did in the last tutorial
# Remember to change all the directory paths to your own ones
%config IPCompleter.greedy=True
%matplotlib notebook
import lib
lib.setup_logger('info', logfile='/home/adebahr/apercal/ipython-notebooks/tutorials/logs/1.log')
import apercal

root - INFO : Logging started!


root - INFO : To see the log in a bash window use the following command:


root - INFO : tail -n +1 -f /home/adebahr/apercal/ipython-notebooks/tutorials/logs/1.log


Setup logger with lib.setup_logger()


Setup logger with lib.setup_logger()


In [2]:
# Load the config file for the tutorial and look at it
prepare = apercal.prepare('/home/adebahr/apercal/ipython-notebooks/tutorials/cfg/1.cfg')
prepare.show(showall=True)

INFO:PREPARE:### Configuration file /home/adebahr/apercal/ipython-notebooks/tutorials/cfg/1.cfg successfully read! ###


INITIAL
	apercaldir = /home/adebahr/apercal
	fluxcal = 3C147.MS
	polcal = 
	target = M51.MS
	basedir = /data/adebahr/apertif/tutorials/T1/
	beam = 00
	rawsubdir = raw
	crosscalsubdir = crosscal
	selfcalsubdir = selfcal
	linesubdir = line
	contsubdir = continuum
	polsubdir = polarisation
PREPARE
	prepare_obsdir_fluxcal = /data/apertif/7070707_3C147/
	prepare_obsdir_polcal = 
	prepare_obsdir_target = /data/apertif/7070708_M51/
	prepare_obsmode = single_element
PREFLAG
	preflag_manualflag = True
	preflag_manualflag_fluxcal = True
	preflag_manualflag_polcal = False
	preflag_manualflag_target = True
	preflag_manualflag_auto = True
	preflag_manualflag_antenna = 
	preflag_manualflag_corr = 
	preflag_manualflag_shadow = True
	preflag_manualflag_baseline = 
	preflag_manualflag_channel = 
	preflag_manualflag_time = 
	preflag_aoflagger = True
	preflag_aoflagger_bandpass = True
	preflag_aoflagger_fluxcal = True
	preflag_aoflagger_polcal = False
	preflag_aoflagger_target = True
	preflag_aoflagger_f

In [3]:
# As you might guess from the name of the dataset we will be working on an M51 dataset, which was observed a while ago.
# This observation used the APERTIF system with 135 MHz of bandwidth in single element mode
# The parameter prepare_obsmode=single_element only copies the central element beam over and does not care about any other beams
# This also means that you can use this parameter to reduce only the central element of a multi-element observation (in case you are not interested in the rest of the field)
# Let the pipeline now create the needed directory structure in the given directory (parameter basedir) and copy over the datasets from the original directories (parameters prepare_obsdir_fluxcal/polcal/target)
prepare.go()

INFO:PREPARE:########## Preparing data for calibration ##########


INFO:PREPARE:### Flux calibrator dataset found. Copying beam 00 to working directory. ###


INFO:PREPARE:### 1 different beams for target field found. ###


INFO:PREPARE:### Copying central element beam to working directory. ###


INFO:PREPARE:### Target dataset(s) copied to working directories! ###


INFO:PREPARE:########## Data prepared for calibration ##########


In [6]:
# You should now have a subdirectory in your basedir + 00 (the number of the beam) + raw (the subdirectory for the raw datasets)
# To execute system tasks within the pipeline you can always import any python module you want and execute functions from them
# Let's look at the contents of the directory using the listdir function of the os module
# You can always access the current initial parameters by using the module name + .[parameter] (We will need that later to fix some issues with the dataset) 
import os
os.listdir(prepare.basedir + prepare.beam + '/' + prepare.rawsubdir)

['3C147.MS', 'M51.MS']

In [7]:
# We now have two datasets in this subdirectory 3C147.MS (calibrator) and M51.MS (target)
# Let's continue with the flagging step
preflag = apercal.preflag('/home/adebahr/apercal/ipython-notebooks/tutorials/cfg/1.cfg')
preflag.show()

PREFLAG
	preflag_manualflag = True
	preflag_manualflag_fluxcal = True
	preflag_manualflag_polcal = False
	preflag_manualflag_target = True
	preflag_manualflag_auto = True
	preflag_manualflag_antenna = 
	preflag_manualflag_corr = 
	preflag_manualflag_shadow = True
	preflag_manualflag_baseline = 
	preflag_manualflag_channel = 
	preflag_manualflag_time = 
	preflag_aoflagger = True
	preflag_aoflagger_bandpass = True
	preflag_aoflagger_fluxcal = True
	preflag_aoflagger_polcal = False
	preflag_aoflagger_target = True
	preflag_aoflagger_fluxcalstrat = fluxcal.rfis
	preflag_aoflagger_polcalstrat = 
	preflag_aoflagger_targetstrat = target.rfis


In [8]:
# We will do manual flagging and after that use the aoflagger to automatically find RFI (parameters preflag_manualflag=True and preflag_aoflagger=True)
# Unfortunately, since we are using some CASA routines, it meses up the logger and crashes the pipeline, if we execute the whole module at once with preflag.go()
# To circumvent this we will execute the individual substeps one by one
# For the manual flagging we will flag the auto-correlations (preflag_manualflag_auto = True) and flag for shadowing (preflag_manualflag_shadow = True)
# In addition we need to create a list of channels to flag the subband edges and the two channels affected by ghosts, which we then insert into the preflag_manualflag_channel parameter

# Total number of channels
nchannel = 11008

# Make lists of the channels which are affected
a = range(0, nchannel, 64) # the subband edges
b = range(1, nchannel, 64)
c = range(63, nchannel, 64)
d = range(16, nchannel, 64) # the two ghosts
e = range(48, nchannel, 64)
# Combine the channel list into one list and convert to a string
l = a + b + c + d + e
preflag.preflag_manualflag_channel = ';'.join(str(ch) for ch in l)

preflag.show()

PREFLAG
	preflag_manualflag = True
	preflag_manualflag_fluxcal = True
	preflag_manualflag_polcal = False
	preflag_manualflag_target = True
	preflag_manualflag_auto = True
	preflag_manualflag_antenna = 
	preflag_manualflag_corr = 
	preflag_manualflag_shadow = True
	preflag_manualflag_baseline = 
	preflag_manualflag_channel = 0;64;128;192;256;320;384;448;512;576;640;704;768;832;896;960;1024;1088;1152;1216;1280;1344;1408;1472;1536;1600;1664;1728;1792;1856;1920;1984;2048;2112;2176;2240;2304;2368;2432;2496;2560;2624;2688;2752;2816;2880;2944;3008;3072;3136;3200;3264;3328;3392;3456;3520;3584;3648;3712;3776;3840;3904;3968;4032;4096;4160;4224;4288;4352;4416;4480;4544;4608;4672;4736;4800;4864;4928;4992;5056;5120;5184;5248;5312;5376;5440;5504;5568;5632;5696;5760;5824;5888;5952;6016;6080;6144;6208;6272;6336;6400;6464;6528;6592;6656;6720;6784;6848;6912;6976;7040;7104;7168;7232;7296;7360;7424;7488;7552;7616;7680;7744;7808;7872;7936;8000;8064;8128;8192;8256;8320;8384;8448;8512;8576;8640;8704;8768;883

In [9]:
# Now execute the manualflag substep
preflag.manualflag()

In [ ]:
# We want to derive a bandpass and apply it to the data before we use aoflagger (parameter preflag_aoflagger_bandpass = True)
# This bandpass will not be used in later stages of the calibration process and helps aoflagger to distinguish between bandpass effects and real RFI better
preflag.aoflagger_bandpass()

TypeError: a float is required

In [11]:
# Execute the aoflagger substep now
# There will be an error message at the end of the step which you can ignore
preflag.aoflagger_flag()

In [12]:
convert = apercal.convert('/home/adebahr/apercal/ipython-notebooks/tutorials/cfg/1.cfg')
convert.show()

INFO:CONVERT:### Configuration file /home/adebahr/apercal/ipython-notebooks/tutorials/cfg/1.cfg successfully read! ###


CONVERT
	convert_ms2uvfits = True
	convert_ms2uvfits_tool_casa_autocorr = False
	convert_uvfits2mir = True
	convert_fluxcal = True
	convert_polcal = False
	convert_target = True


In [13]:
# The convert step coverts the datasets from MS to UVFITS (convert_ms2uvfits = True) and then to MIRIAD format (convert_uvfits2mir = True)
# We do not want to carry the auto-correlations over (convert_ms2uvfits_tool_casa_autocorr = False)
# The converted datasets are saved in the crosscal subdirectory
convert.go()

INFO:CONVERT:########## FILE CONVERSION started ##########


INFO:CONVERT:### Starting conversion from MS to UVFITS format ###


INFO:CONVERT:### Converted MS file 3C147.MS to UVFITS format! ###


INFO:CONVERT:### Converted MS file /data/adebahr/apertif/tutorials/T1/00/raw/M51.MS to UVFITS format! ###


INFO:CONVERT:### Conversion from MS to UVFITS format done! ###


INFO:CONVERT:### Starting conversion from UVFITS to MIRIAD format ###


INFO:CONVERT:### Converted UVFITS file 3C147.MS to MIRIAD format! ###


INFO:CONVERT:### Converted UVFITS file /data/adebahr/apertif/tutorials/T1/00/crosscal/M51.UVFITS to MIRIAD format! ###


INFO:CONVERT:### Conversion from UVFITS to MIRIAD format done! ###


INFO:CONVERT:########## FILE CONVERSION done ##########


In [14]:
# Since this is an old dataset (anything after 21.12.17 should be fine), we need to recompute the uv-coordinates
# This can be done manually using the task uvedit in MIRIAD
# We change the pointing centre by an infinitesimal amount (0.00001s) to force uvedit to recompute them

import os

# Initialise the task
uvedit = lib.miriad('uvedit')

# Execute the task for the calibrator
uvedit.vis = convert.crosscaldir + '/' + convert.fluxcal.rstrip('MS') + 'mir'
uvedit.ra = 0.00001
uvedit.out = convert.crosscaldir + '/' + convert.fluxcal.rstrip('.MS') + '_uvedit.mir'
uvedit.go()

# Execute the task for the target
uvedit.vis = convert.crosscaldir + '/' + convert.target.rstrip('MS') + 'mir'
uvedit.ra = 0.00001
uvedit.out = convert.crosscaldir + '/' + convert.target.rstrip('.MS') + '_uvedit.mir'
uvedit.go()

# Remove the obsolete files from the directories
os.system('rm -rf ' + convert.crosscaldir + '/' + convert.fluxcal.rstrip('MS') + 'mir')
os.system('rm -rf ' + convert.crosscaldir + '/' + convert.target.rstrip('MS') + 'mir')
os.system('mv ' + convert.crosscaldir + '/' + convert.fluxcal.rstrip('.MS') + '_uvedit.mir ' + convert.crosscaldir + '/' + convert.fluxcal.rstrip('.MS') + '.mir')
os.system('mv ' + convert.crosscaldir + '/' + convert.target.rstrip('.MS') + '_uvedit.mir ' + convert.crosscaldir + '/' + convert.target.rstrip('.MS') + '.mir')

0

In [15]:
# Since this is an old dataset (anything after 21.12.17 should be fine), we need to flip the phases
# This can be done manually using the task uvrot in MIRIAD

import os

# Initialise the task
uvrot = lib.miriad('uvrot')

# Execute the task for the calibrator
uvrot.vis = convert.crosscaldir + '/' + convert.fluxcal.rstrip('.MS') + '.mir'
uvrot.angle = 180
uvrot.out = convert.crosscaldir + '/' + convert.fluxcal.rstrip('.MS') + '_uvrot.mir'
uvrot.go()

# Execute the task for the target
uvrot.vis = convert.crosscaldir + '/' + convert.target.rstrip('.MS') + '.mir'
uvrot.angle = 180
uvrot.out = convert.crosscaldir + '/' + convert.target.rstrip('.MS') + '_uvrot.mir'
uvrot.go()

# Remove the obsolete files from the directories
os.system('rm -rf ' + convert.crosscaldir + '/' + convert.fluxcal.rstrip('MS') + 'mir')
os.system('rm -rf ' + convert.crosscaldir + '/' + convert.target.rstrip('MS') + 'mir')
os.system('mv ' + convert.crosscaldir + '/' + convert.fluxcal.rstrip('.MS') + '_uvrot.mir ' + convert.crosscaldir + '/' + convert.fluxcal.rstrip('.MS') + '.mir')
os.system('mv ' + convert.crosscaldir + '/' + convert.target.rstrip('.MS') + '_uvrot.mir ' + convert.crosscaldir + '/' + convert.target.rstrip('.MS') + '.mir')

0

In [16]:
ccal = apercal.ccal('/home/adebahr/apercal/ipython-notebooks/tutorials/cfg/1.cfg')
ccal.show()

INFO:CROSSCAL:### Configuration file /home/adebahr/apercal/ipython-notebooks/tutorials/cfg/1.cfg successfully read! ###


CROSSCAL
	crosscal_bandpass = True
	crosscal_delay = True
	crosscal_polarisation = False
	crosscal_transfer_to_target = True


In [17]:
# Crosscal derives the gains and bandpass (crosscal_bandpass = True) from the calibrator and applies them to the target field (crosscal_transfer_to_target = True)
# We also want to solve for the delays. This will work better for recent datasets (since we have the cable delay compensation in place), but still might need to be done (the delay compensation can only be sampled in increments of 5ns)
ccal.go()

INFO:CROSSCAL:########## Starting CROSS CALIBRATION ##########


INFO:CROSSCAL:### Bandpass calibration on the flux calibrator data started ###


INFO:CROSSCAL:### Bandpass calibration on the flux calibrator data done ###


INFO:CROSSCAL:### No polarisation calibration done! ###


INFO:CROSSCAL:### Copying calibrator solutions to target dataset ###


...

INFO:CROSSCAL:# Polarisation calibration solutions (leakage, angle) not found #


INFO:CROSSCAL:# Copying calibrator solutions to 1 beams! #


INFO:CROSSCAL:# Calibrator solutions copied to beam 00! #


INFO:CROSSCAL:### All solutions copied to target data set(s) ###


INFO:CROSSCAL:########## CROSS CALIBRATION done ##########


In [19]:
scal = apercal.scal('/home/adebahr/apercal/ipython-notebooks/tutorials/cfg/1.cfg')
scal.show()

INFO:SELFCAL:### Configuration file /home/adebahr/apercal/ipython-notebooks/tutorials/cfg/1.cfg successfully read! ###


SELFCAL
	selfcal_image_imsize = 1025
	selfcal_image_cellsize = 8
	selfcal_splitdata = True
	selfcal_splitdata_chunkbandwidth = 0.02
	selfcal_splitdata_channelbandwidth = 0.001
	selfcal_flagantenna = 
	selfcal_flagline = True
	selfcal_flagline_sigma = 0.5
	selfcal_parametric = False
	selfcal_parametric_skymodel_radius = 0.5
	selfcal_parametric_skymodel_cutoff = 0.8
	selfcal_parametric_skymodel_distance = 30
	selfcal_parametric_solint = 10
	selfcal_parametric_uvmin = 0.5
	selfcal_parametric_uvmax = 1000
	selfcal_parametric_amp = False
	selfcal_standard_majorcycle = 3
	selfcal_standard_majorcycle_function = square
	selfcal_standard_minorcycle = 5
	selfcal_standard_minorcycle_function = square
	selfcal_standard_c0 = 10.0
	selfcal_standard_c1 = 5.0
	selfcal_standard_minorcycle0_dr = 3.0
	selfcal_standard_drinit = 25
	selfcal_standard_dr0 = 2.0
	selfcal_standard_nsigma = 50
	selfcal_standard_uvmin = [0.0, 0.0, 0.0]
	selfcal_standard_uvmax = [3000, 3000, 3000]
	selfcal_standard_solint = [5, 3

In [20]:
# For the self-calibration we are using an image size of 1025 pixels (selfcal_image_imsize = 1025) with a cell size of 8 arcseconds (selfcal_image_cellsize = 8)
# We split the data (selfcal_splitdata = True) in chunks of 0.02 GHz (selfcal_splitdata_chunkbandwidth = 0.02) and average to a channel bandwidth of 0.001 GHz (selfcal_splitdata_channelbandwidth = 0.001)
# The algorithm will split the observed band uniformly between the chunks. The used chunk and chanel bandwidth will therefore be close but not exactly the given ones
# We do not do any parametric self-calibration (selfcal_parametric = False) since our observed target field is dominated by diffuse emission
# We use 3 majorcycles (selfcal_standard_majorcycle = 3) with 5 minorcycles each (selfcal_standard_minorcycle = 5)
# Both types of cycles have a square function in calculate the maximum increment in dynamic range between the cycles (selfcal_standard_majorcycle_function = square/selfcal_standard_minorcycle_function = square)
# We use the complete uv-range for every cycle (selfcal_standard_uvmin = [0.0, 0.0, 0.0]/selfcal_standard_uvmax = [3000, 3000, 3000])
# and solution intervals of 5,3, and 0.5 minutes for the three major cycles (selfcal_standard_solint = [5, 3, 0.5])
# We do not use any amplitude calibration (selfcal_standard_amp = False). You can experiment with options True (always use amplitude calibration) or 'auto' (use amplitude calibration if the clean components reach a flux of selfcal_standard_amp_auto_limit Jy)
# Only one solution is used for each individual chunk (selfcal_standard_nfbin = 1)
scal.go()

INFO:SELFCAL:########## Starting SELF CALIBRATION ##########


INFO:SELFCAL:# Creating directory /data/adebahr/apertif/tutorials/T1/00/selfcal #


INFO:SELFCAL:# Moved to directory /data/adebahr/apertif/tutorials/T1/00/selfcal #


INFO:SELFCAL:### Splitting of target data into individual frequency chunks started ###


INFO:SELFCAL:# Applying calibrator solutions to target data before averaging #


...

INFO:SELFCAL:# Doing self-calibration with uvmin=0.0, uvmax=3000, solution interval=0.5 minutes for major cycle 02 #


INFO:SELFCAL:# Major self-calibration cycle 2 for frequency chunk 07 finished #


INFO:SELFCAL:# Standard self-calibration routine for chunk 07 finished #


INFO:SELFCAL:### Standard self calibration routine finished ###


INFO:SELFCAL:########## SELF CALIBRATION done ##########


In [23]:
# All frequency chunks were calibrated and it is now time to create a final continuum image
continuum = apercal.continuum('/home/adebahr/apercal/ipython-notebooks/tutorials/cfg/1.cfg')
continuum.show()

INFO:CONTINUUM:### Configuration file /home/adebahr/apercal/ipython-notebooks/tutorials/cfg/1.cfg successfully read! ###


CONTINUUM
	continuum_image_imsize = 1025
	continuum_image_cellsize = 8
	continuum_image_centre = 
	continuum_image_restorbeam = 
	continuum_image_convolbeam = 45,45,0.0
	continuum_mode = stack
	continuum_majorcycle_function = square
	continuum_minorcycle_function = square
	continuum_minorcycle0_dr = 3.0
	continuum_minorcycle = 5
	continuum_c0 = 10.0
	continuum_c1 = 5.0
	continuum_drinit = 25
	continuum_dr0 = 2.0
	continuum_nsigma = 30
	continuum_robust = 


In [24]:
# We are going to use the same image size and cellsize as for the self-calibration (continuum_image_imsize = 1025/continuum_image_cellsize = 8)
# Since we are stacking all the final images of the individual chunks in the end (continuum_mode = stack) we want to convolve with a commen beam (continuum_image_convolbeam = 45,45,0.0)
# We use the same paramaters as for the self-calibration for the final cleaning procedure
continuum.go()

INFO:CONTINUUM:########## Starting CONTINUUM IMAGING ##########


INFO:CONTINUUM:### Starting deep continuum imaging of full dataset ###


INFO:CONTINUUM:# Creating directory /data/adebahr/apertif/tutorials/T1/00/continuum #


INFO:CONTINUUM:# Moved to directory /data/adebahr/apertif/tutorials/T1/00/continuum #


INFO:CONTINUUM:### Creating individual deep images from frequency chunks ###


...

INFO:CONTINUUM:# Moved to directory /data/adebahr/apertif/tutorials/T1/00/continuum/stack #


INFO:CONTINUUM:# Convolving final continuum images to major/minor fwhm 45,45and angle 0.0


INFO:CONTINUUM:### Final deep continuum image is /data/adebahr/apertif/tutorials/T1/00/continuum/M51_stack ###


INFO:CONTINUUM:### Deep continuum imaging of full dataset done ###


INFO:CONTINUUM:########## CONTINUUM IMAGING done ##########


In [27]:
# Since the deep continuum images are ready now, we can use this model to subtract the continuum and image the line channels
# In case of a non-existent continuum model from the continuum step, the line module would automatically image the continuum beforehand and use it for subtraction
line = apercal.line('/home/adebahr/apercal/ipython-notebooks/tutorials/cfg/1.cfg')
line.show()

INFO:LINE:### Configuration file /home/adebahr/apercal/ipython-notebooks/tutorials/cfg/1.cfg successfully read! ###


LINE
	line_splitdata = True
	line_splitdata_chunkbandwidth = 0.02
	line_splitdata_channelbandwidth = 1.2e-05
	line_transfergains = True
	line_subtract = True
	line_subtract_mode = uvmodel
	line_subtract_mode_uvmodel_majorcycle_function = square
	line_subtract_mode_uvmodel_minorcycle_function = square
	line_subtract_mode_uvmodel_minorcycle = 5
	line_subtract_mode_uvmodel_c0 = 10.0
	line_subtract_mode_uvmodel_c1 = 5.0
	line_subtract_mode_uvmodel_drinit = 25
	line_subtract_mode_uvmodel_dr0 = 2.0
	line_subtract_mode_uvmodel_nsigma = 30
	line_subtract_mode_uvmodel_imsize = 513
	line_subtract_mode_uvmodel_cellsize = 10
	line_subtract_mode_uvmodel_minorcycle0_dr = 3.0
	line_image = True
	line_image_channels = 9650,9800
	line_image_imsize = 1025
	line_image_cellsize = 8
	line_image_centre = 
	line_image_robust = -2
	line_image_ratio_limit = 1.03
	line_image_c0 = 10.0
	line_image_c1 = 5.0
	line_image_nsigma = 20
	line_image_minorcycle0_dr = 3.0
	line_image_dr0 = 2.0
	line_image_restorbeam = 
	l

In [28]:
# The line module is using the originally converted MIRIAD data files, splits and compresses them to the desired channel bandwidth (parameters: line_splitdata_chunkbandwidth and line_splitdata_channelbandwidth)
# Then it transfers the gains from the self calibration (line_transfergains) and applies them before subtracting the continuum model (line_subtract = True/line_subtract_mode = uvmodel)
# Imaging of the line cube (line_image = True) is done with the same image size (line_image_imsize = 1025) and cell size (line_image_cellsize = 8) as for the continuum
# We a robust weighting of -2 for the imaging (line_image_robust = -2) and clean channels where the ratio between the maximum and minimum pixel value is larger than 1.05 (line_image_ratio_limit = 1.05)
# We only image the channel range between channel 9650 and 9800 (line_image_channels = 9650,9800) and all images are covolved to 45 arcseconds (line_image_convolbeam = 45,45,0.0)
line.go()

INFO:LINE:########## Starting CONTINUUM SUBTRACTION ##########


INFO:LINE:# Creating directory /data/adebahr/apertif/tutorials/T1/00/line #


INFO:LINE:# Moved to directory /data/adebahr/apertif/tutorials/T1/00/line #


INFO:LINE:### Splitting of target data into individual frequency chunks started ###


INFO:LINE:# Applying calibrator solutions to target data before averaging #


...

INFO:LINE:# Finished processing channel 09746/11007. #


INFO:LINE:# Emission found in channel 09747. Cleaning! #


INFO:LINE:# Finished processing channel 09747/11007. #


INFO:LINE:# Emission found in channel 09748. Cleaning! #


KeyboardInterrupt: 